# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [20]:
# import libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
# load data from database
engine = create_engine('sqlite:///project_db.db')
df = pd.read_sql_table(con=engine, table_name='table1')
df.dropna(axis=0, subset=df.columns[4:],inplace=True)
X = df['message']
y = df.iloc[:,4:]
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
def remove_all_zero_column(df):
    return df.loc[:,(df!=0).any(axis=0)]    
y = remove_all_zero_column(y)
y.mean()

related                   0.774526
request                   0.170947
offer                     0.004541
aid_related               0.415080
medical_help              0.079635
medical_products          0.050139
search_and_rescue         0.027626
security                  0.017972
military                  0.032816
water                     0.063876
food                      0.111802
shelter                   0.088488
clothing                  0.015492
money                     0.023047
missing_people            0.011409
refugees                  0.033426
death                     0.045637
other_aid                 0.131568
infrastructure_related    0.065059
transport                 0.045904
buildings                 0.050941
electricity               0.020376
tools                     0.006067
hospitals                 0.010799
shops                     0.004579
aid_centers               0.011791
other_infrastructure      0.043920
weather_related           0.278628
floods              

### 2. Write a tokenization function to process your text data

In [23]:
def tokenize(text):
    words = word_tokenize(text)
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [24]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=LogisticRegression()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 77)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [26]:
y_pred = pd.DataFrame(y_pred, columns=y_test.columns)
y_pred.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
def print_classification_report(test, pred, col_ind):
    print(classification_report(test.iloc[:,col_ind], pred.iloc[:,col_ind]))

col_number = y_train.shape[1]
for i in range(col_number):
    print_classification_report(y_test, y_pred, i)

             precision    recall  f1-score   support

        0.0       0.39      0.03      0.06      1247
        1.0       0.76      0.99      0.86      3956
        2.0       0.00      0.00      0.00        39

avg / total       0.67      0.75      0.66      5242

             precision    recall  f1-score   support

        0.0       0.84      0.97      0.90      4352
        1.0       0.48      0.12      0.20       890

avg / total       0.78      0.83      0.78      5242

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      5219
        1.0       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      5242

             precision    recall  f1-score   support

        0.0       0.59      0.81      0.69      3024
        1.0       0.49      0.25      0.33      2218

avg / total       0.55      0.57      0.53      5242

             precision    recall  f1-score   support

        0.0       0.92      1.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [28]:
parameters = {
    'clf__estimator__penalty':['l1','l2'],
    'clf__estimator__C':[0.01, 0.1, 0.3, 0.5, 1.5,3,5]
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
cv.fit(X=X_train, y=y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__penalty': ['l1', 'l2'], 'clf__estimator__C': [0.01, 0.1, 0.3, 0.5, 1.5, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [30]:
best_model = cv.best_estimator_

In [31]:
y_pred_cv = best_model.predict(X_test)

In [32]:
y_pred_cv = pd.DataFrame(y_pred_cv, columns=y_test.columns)
for i in range(col_number):
    print_classification_report(y_test, y_pred_cv, i)

             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00      1247
        1.0       0.75      1.00      0.86      3956
        2.0       0.00      0.00      0.00        39

avg / total       0.57      0.75      0.65      5242

             precision    recall  f1-score   support

        0.0       0.83      1.00      0.91      4352
        1.0       0.00      0.00      0.00       890

avg / total       0.69      0.83      0.75      5242

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      5219
        1.0       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      5242

             precision    recall  f1-score   support

        0.0       0.58      1.00      0.73      3024
        1.0       0.00      0.00      0.00      2218

avg / total       0.33      0.58      0.42      5242

             precision    recall  f1-score   support

        0.0       0.92      1.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [33]:
cv.best_params_

{'clf__estimator__C': 0.01, 'clf__estimator__penalty': 'l1'}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [34]:
upd_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))
])

upd_pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 9. Export your model as a pickle file

In [35]:
pickle.dump(best_model, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.